In [51]:
workspace = 'local' # local | remote | colab

# Libraries

In [52]:
import os
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from pprint import pprint

from src.utils import (
    # Old utils
    print_h, eval_person_majority_voting,

    # New utils
    set_seed, get_device, init_metrics, update_metrics, save_metrics_to_json,
)
from src.models import RNNInceptionTime

# Config

In [53]:
# Project config
seed = 69
set_seed(seed)
device = get_device()
print("Device:", device)

# Model config
k_fold_dir = (
    # On remote server
    ''
) if workspace == 'remote' else (
    # On local machine
    'data/preprocessed/Ga_k10_w500_s500_v20250430232239'
)
model_dir = (
    # On remote server
    ''
) if workspace == 'remote' else (
    # On local machine
    'checkpoints/RNNInceptionTime_Ga_k10_w500_s500_e1_v20250430215415'
)
model_name = model_dir.split('/')[-1]
study = model_name.split('RNNInceptionTime_')[-1].rsplit('_')[0]
print("Dataset study:", study)

# Evaluation config
batch_size = 8
n_feat = 16
n_class = 4
window_size = 500
max_vgrf_data_len = 25_000

model_paths = sorted([model_dir+'/'+f for f in os.listdir(model_dir) if f.endswith('.pth')])
k_fold = len([f for f in os.listdir(model_dir) if f.endswith('.pth')])

general_metrics_dir = f'evaluations/{model_name}/_general_metrics'

Random seed: 69
Device: mps
Dataset study: Ga


# Evaluation

In [54]:
metrics = {
    'person_majority_voting': init_metrics(['acc', 'f1', 'precision', 'recall', 'cm']),
    # 'person_severity_voting': init_metrics(['acc', 'f1', 'precision', 'recall', 'cm']),
    # 'person_max_severity': init_metrics(['acc', 'f1', 'precision', 'recall', 'cm']),
    # 'window': init_metrics(['acc', 'f1', 'precision', 'recall', 'cm']),
}

study_label_map = {
    'Ga': 0,
    'Ju': 1,
    'Si': 2,
}

for i_fold in range(k_fold):
    print_h(f"FOLD-{i_fold+1}", 128)

    X_train_window_GaJuSi = torch.empty(0, window_size, n_feat).float()
    y_train_window_GaJuSi = torch.empty(0).long()
    study_labels_train_window_GaJuSi = torch.empty(0).long()
    
    X_val_window_GaJuSi = torch.empty(0, window_size, n_feat).float()
    y_val_window_GaJuSi = torch.empty(0).long()
    study_labels_val_window_GaJuSi = torch.empty(0).long()

    X_test_window_GaJuSi = torch.empty(0, window_size, n_feat).float()
    y_test_window_GaJuSi = torch.empty(0).long()
    study_labels_test_window_GaJuSi = torch.empty(0).long()

    X_val_person_GaJuSi = torch.empty(0, max_vgrf_data_len, n_feat).float()
    y_val_person_GaJuSi = torch.empty(0).long()
    # study_labels_val_person_GaJuSi = torch.empty(0).long()

    X_test_person_GaJuSi = torch.empty(0, max_vgrf_data_len, n_feat).float()
    y_test_person_GaJuSi = torch.empty(0).long()
    # study_labels_test_person_GaJuSi = torch.empty(0).long()

    print_h(f"EXPERT-{study} MODEL", 96)

    fold_i_dir_name = os.listdir(k_fold_dir)[i_fold]
    fold_i_dir = os.path.join(k_fold_dir, fold_i_dir_name)

    X_train_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'X_train_window.npy'))).float()
    y_train_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'y_train_window.npy'))).long()
    study_labels_train_window = torch.tensor([study_label_map[study]] * len(y_train_window)).long()
    X_train_window_GaJuSi = torch.cat((X_train_window_GaJuSi, X_train_window), dim=0)
    y_train_window_GaJuSi = torch.cat((y_train_window_GaJuSi, y_train_window), dim=0)
    study_labels_train_window_GaJuSi = torch.cat((study_labels_train_window_GaJuSi, study_labels_train_window), dim=0)

    X_val_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'X_val_window.npy'))).float()
    y_val_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'y_val_window.npy'))).long()
    study_labels_val_window = torch.tensor([study_label_map[study]] * len(y_val_window)).long()
    X_val_window_GaJuSi = torch.cat((X_val_window_GaJuSi, X_val_window), dim=0)
    y_val_window_GaJuSi = torch.cat((y_val_window_GaJuSi, y_val_window), dim=0)
    study_labels_val_window_GaJuSi = torch.cat((study_labels_val_window_GaJuSi, study_labels_val_window), dim=0)

    X_test_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'X_test_window.npy'))).float()
    y_test_window = torch.tensor(np.load(os.path.join(fold_i_dir, f'y_test_window.npy'))).long()
    study_labels_test_window = torch.tensor([study_label_map[study]] * len(y_test_window)).long()
    X_test_window_GaJuSi = torch.cat((X_test_window_GaJuSi, X_test_window), dim=0)
    y_test_window_GaJuSi = torch.cat((y_test_window_GaJuSi, y_test_window), dim=0)
    study_labels_test_window_GaJuSi = torch.cat((study_labels_test_window_GaJuSi, study_labels_test_window), dim=0)

    X_val_person = torch.tensor(np.load(os.path.join(fold_i_dir, f'X_val_person.npy'))).float()
    y_val_person = torch.tensor(np.load(os.path.join(fold_i_dir, f'y_val_person.npy'))).long()
    X_val_person_GaJuSi = torch.cat((X_val_person_GaJuSi, X_val_person), dim=0)
    y_val_person_GaJuSi = torch.cat((y_val_person_GaJuSi, y_val_person), dim=0)

    X_test_person = torch.tensor(np.load(os.path.join(fold_i_dir, f'X_test_person.npy'))).float()
    y_test_person = torch.tensor(np.load(os.path.join(fold_i_dir, f'y_test_person.npy'))).long()
    X_test_person_GaJuSi = torch.cat((X_test_person_GaJuSi, X_test_person), dim=0)
    y_test_person_GaJuSi = torch.cat((y_test_person_GaJuSi, y_test_person), dim=0)

    train_window_dataset = TensorDataset(X_train_window, y_train_window)
    val_window_dataset = TensorDataset(X_val_window, y_val_window)
    test_window_dataset = TensorDataset(X_test_window, y_test_window)
    
    val_person_dataset = TensorDataset(X_val_person, y_val_person)
    test_person_dataset = TensorDataset(X_test_person, y_test_person)

    train_dataloader = DataLoader(train_window_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_window_dataset, batch_size=batch_size, shuffle=False)
    test_dataloader = DataLoader(test_window_dataset, batch_size=batch_size, shuffle=False)
    
    # Load pretrained expert model
    model = RNNInceptionTime(c_in=n_feat, c_out=n_class, seq_len=window_size, bidirectional=True).to(device)
    model_i_path = model_paths[i_fold]
    model.load_state_dict(torch.load(model_i_path, map_location=device))

    print_h("EVALUATION ON PERSON DATA BY MAJORITY VOTING", 64)
    (
        _, 
        acc_person_majority_voting, 
        f1_person_majority_voting, 
        precision_person_majority_voting, 
        recall_person_majority_voting, 
        cm_person_majority_voting, 
        *_
    ) = eval_person_majority_voting(
        model, 
        val_person_dataset, 
        criterion=None, 
        average='weighted',
        window_size=window_size, 
        debug=False
    )
    print("acc:", acc_person_majority_voting)
    print("f1:", f1_person_majority_voting)
    print("precision:", precision_person_majority_voting)
    print("recall:", recall_person_majority_voting)
    print("cm:\n", np.array(cm_person_majority_voting))
    print()

    # metrics = update_metrics(metrics, {
    #     'acc': acc_person_majority_voting,
    #     'f1': f1_person_majority_voting,
    #     'precision': precision_person_majority_voting,
    #     'recall': recall_person_majority_voting,
    #     'cm': cm_person_majority_voting,
    # })

    in_metrics = {
        'person_majority_voting': {
            'acc': acc_person_majority_voting,
            'f1': f1_person_majority_voting,
            'precision': precision_person_majority_voting,
            'recall': recall_person_majority_voting,
            'cm': cm_person_majority_voting,
        },
        # 'person_severity_voting': {
        #     'acc': acc_person_severity_voting,
        #     'f1': f1_person_severity_voting,
        #     'precision': precision_person_severity_voting,
        #     'recall': recall_person_severity_voting,
        #     'cm': cm_person_severity_voting,
        # },
        # 'person_max_severity': {
        #     'acc': acc_person_max_severity,
        #     'f1': f1_person_max_severity,
        #     'precision': precision_person_max_severity,
        #     'recall': recall_person_max_severity,
        #     'cm': cm_person_max_severity,
        # },
        # 'window': {
        #     'acc': acc_window,
        #     'f1': f1_window,
        #     'precision': precision_window,
        #     'recall': recall_window,
        #     'cm': cm_window,
        # },
    }

    for metric_type in in_metrics.keys():
        update_metrics(metrics[metric_type], in_metrics[metric_type])

    # DEBUG: Test for only one fold
    # break

                                                             FOLD-1                                                             
                                        EXPERT-Ga MODEL                                         
          EVALUATION ON PERSON DATA BY MAJORITY VOTING          
acc: 0.875
f1: 0.8672619047619048
precision: 0.90625
recall: 0.875
cm:
 [[5 0 0 0]
 [0 6 0 0]
 [0 1 2 0]
 [0 1 0 1]]

                                                             FOLD-2                                                             
                                        EXPERT-Ga MODEL                                         
          EVALUATION ON PERSON DATA BY MAJORITY VOTING          
acc: 0.875
f1: 0.8214285714285714
precision: 0.78125
recall: 0.875
cm:
 [[5 0 0 0]
 [0 6 0 0]
 [0 0 3 0]
 [0 2 0 0]]

                                                             FOLD-3                                                             
                                        EXPERT-Ga MOD

## Metrics

In [56]:
print_h("METRICS", 128)
save_metrics_to_json(metrics, general_metrics_dir, f'_{study}.json')
pprint(metrics, sort_dicts=False)
print()
print("Evaluation metrics is saved in:", general_metrics_dir)

                                                            METRICS                                                             
{'person_majority_voting': {'acc': {'folds': [0.875,
                                              0.875,
                                              0.6875,
                                              0.75,
                                              0.8125,
                                              0.9375,
                                              0.8125,
                                              0.75,
                                              0.8125,
                                              0.8125],
                                    'avg': 0.8125,
                                    'std': 0.06846531968814576},
                            'f1': {'folds': [0.8672619047619048,
                                             0.8214285714285714,
                                             0.6489583333333333,
                         